In [1]:
# Audio: MSP-Conversation_0021_2_007 | LINE: 12076 | DOMINANCE

In [2]:
import os
os.chdir(os.path.abspath(".."))


import librosa
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings('ignore')

from src.dataload import load_audio_data, audio_select_mean_vote
from src.emotion_translation import ekman_emotion
from src.metricas import classical_features, get_training_features_simple

In [3]:
df_annotations = pd.read_excel("data/annotations_2.xlsx")
df_training_audios = df_annotations[df_annotations['Type'] == 'Train'][['Audio_Name','Part_Num']].drop_duplicates().reset_index(drop = True)

In [8]:
len(timestamps)

5208142

In [10]:
time_index

0            0.000000
1            0.000063
2            0.000125
3            0.000188
4            0.000250
              ...    
5373901    325.599600
5373902    325.601800
5373903    325.602300
5373904    325.620600
5373905    332.008900
Name: Time, Length: 5373906, dtype: float64

In [13]:
df_train

,Time,Data,Arousal,Dominance,Valence,Emotion
0,0.000000,-0.010864,-0.002000,0.000000,-0.0025,disgust
1,0.000063,-0.012909,-0.002000,0.000000,-0.0025,disgust
2,0.000125,-0.009735,-0.002000,0.000000,-0.0025,disgust
3,0.000188,-0.004852,-0.002000,0.000000,-0.0025,disgust
4,0.000250,-0.006348,-0.002000,0.000000,-0.0025,disgust
...,...,...,...,...,...,...
4606746,325.509000,-0.000244,13.151667,27.243333,-13.9500,disgust
4606747,325.513100,-0.000244,13.151667,27.243333,-13.6850,disgust
4606748,325.566300,-0.000244,12.888333,27.243333,-13.6850,disgust
4606749,325.578900,-0.000244,12.888333,27.700000,-13.6850,disgust


In [4]:
start_time = time.time()


df_features_complete = pd.DataFrame()

# Main loop for audio data
for index, row in df_training_audios.iterrows():
    
    print('Audio:', row['Audio_Name'], ' | Part:', row['Part_Num'])
    print(index , '/', df_training_audios.index.max())
    # Input
    data, timestamps, sr = load_audio_data(df_annotations, part_num = row['Part_Num'], audio_name = row['Audio_Name'])
    df_input = pd.DataFrame(data = np.stack([timestamps, data]).T, columns = ['Time','Data'])
    
    # Labels
    df_label = audio_select_mean_vote(df_annotations,  part_num = row['Part_Num'], audio_name = row['Audio_Name'])
    df_label['Emotion'] = df_label.apply(lambda row : ekman_emotion(row['Valence'], row['Arousal'], row['Dominance']), axis = 1)
    
    time_index = pd.concat([df_label['Time'],df_input['Time']], ignore_index = True).drop_duplicates().sort_values().reset_index(drop = True)
    df_train = pd.merge(time_index, df_input, how = 'left', on = 'Time')
    df_train = pd.merge(df_train, df_label, how = 'left', on = 'Time')
    
    df_train = df_train.fillna(method='ffill').fillna(method='bfill')
    df_train = df_train.set_index('Time').drop_duplicates().reset_index()
    
    paramaters = {'sr' : sr}
    df_features = get_training_features_simple(classical_features, paramaters, frame_duration = 2.5, start = 0, step = 0.5, df = df_train)
    
    df_features_complete = pd.concat([df_features_complete , df_features])
    
end_time = time.time()
print(end_time - start_time)    

Audio: MSP-Conversation_0021.wav  | Part: 1
0 / 428
0.5 / 306.122
1.0 / 306.122
1.5 / 306.122
2.0 / 306.122
2.5 / 306.122
3.0 / 306.122
3.5 / 306.122
4.0 / 306.122
4.5 / 306.122
5.0 / 306.122
5.5 / 306.122
6.0 / 306.122
6.5 / 306.122
7.0 / 306.122
7.5 / 306.122
8.0 / 306.122
8.5 / 306.122
9.0 / 306.122
9.5 / 306.122
10.0 / 306.122
10.5 / 306.122
11.0 / 306.122
11.5 / 306.122
12.0 / 306.122
12.5 / 306.122
13.0 / 306.122
13.5 / 306.122
14.0 / 306.122
14.5 / 306.122
15.0 / 306.122
15.5 / 306.122
16.0 / 306.122
16.5 / 306.122
17.0 / 306.122
17.5 / 306.122
18.0 / 306.122
18.5 / 306.122
19.0 / 306.122
19.5 / 306.122
20.0 / 306.122
20.5 / 306.122
21.0 / 306.122
21.5 / 306.122
22.0 / 306.122
22.5 / 306.122
23.0 / 306.122
23.5 / 306.122
24.0 / 306.122
24.5 / 306.122
25.0 / 306.122
25.5 / 306.122
26.0 / 306.122
26.5 / 306.122
27.0 / 306.122
27.5 / 306.122
28.0 / 306.122
28.5 / 306.122
29.0 / 306.122
29.5 / 306.122
30.0 / 306.122
30.5 / 306.122
31.0 / 306.122
31.5 / 306.122
32.0 / 306.122
32.5 / 

262.0 / 306.122
262.5 / 306.122
263.0 / 306.122
263.5 / 306.122
264.0 / 306.122
264.5 / 306.122
265.0 / 306.122
265.5 / 306.122
266.0 / 306.122
266.5 / 306.122
267.0 / 306.122
267.5 / 306.122
268.0 / 306.122
268.5 / 306.122
269.0 / 306.122
269.5 / 306.122
270.0 / 306.122
270.5 / 306.122
271.0 / 306.122
271.5 / 306.122
272.0 / 306.122
272.5 / 306.122
273.0 / 306.122
273.5 / 306.122
274.0 / 306.122
274.5 / 306.122
275.0 / 306.122
275.5 / 306.122
276.0 / 306.122
276.5 / 306.122
277.0 / 306.122
277.5 / 306.122
278.0 / 306.122
278.5 / 306.122
279.0 / 306.122
279.5 / 306.122
280.0 / 306.122
280.5 / 306.122
281.0 / 306.122
281.5 / 306.122
282.0 / 306.122
282.5 / 306.122
283.0 / 306.122
283.5 / 306.122
284.0 / 306.122
284.5 / 306.122
285.0 / 306.122
285.5 / 306.122
286.0 / 306.122
286.5 / 306.122
287.0 / 306.122
287.5 / 306.122
288.0 / 306.122
288.5 / 306.122
289.0 / 306.122
289.5 / 306.122
290.0 / 306.122
290.5 / 306.122
291.0 / 306.122
291.5 / 306.122
292.0 / 306.122
292.5 / 306.122
293.0 / 

206.5 / 296.2987
207.0 / 296.2987
207.5 / 296.2987
208.0 / 296.2987
208.5 / 296.2987
209.0 / 296.2987
209.5 / 296.2987
210.0 / 296.2987
210.5 / 296.2987
211.0 / 296.2987
211.5 / 296.2987
212.0 / 296.2987
212.5 / 296.2987
213.0 / 296.2987
213.5 / 296.2987
214.0 / 296.2987
214.5 / 296.2987
215.0 / 296.2987
215.5 / 296.2987
216.0 / 296.2987
216.5 / 296.2987
217.0 / 296.2987
217.5 / 296.2987
218.0 / 296.2987
218.5 / 296.2987
219.0 / 296.2987
219.5 / 296.2987
220.0 / 296.2987
220.5 / 296.2987
221.0 / 296.2987
221.5 / 296.2987
222.0 / 296.2987
222.5 / 296.2987
223.0 / 296.2987
223.5 / 296.2987
224.0 / 296.2987
224.5 / 296.2987
225.0 / 296.2987
225.5 / 296.2987
226.0 / 296.2987
226.5 / 296.2987
227.0 / 296.2987
227.5 / 296.2987
228.0 / 296.2987
228.5 / 296.2987
229.0 / 296.2987
229.5 / 296.2987
230.0 / 296.2987
230.5 / 296.2987
231.0 / 296.2987
231.5 / 296.2987
232.0 / 296.2987
232.5 / 296.2987
233.0 / 296.2987
233.5 / 296.2987
234.0 / 296.2987
234.5 / 296.2987
235.0 / 296.2987
235.5 / 296.29

168.5 / 303.071
169.0 / 303.071
169.5 / 303.071
170.0 / 303.071
170.5 / 303.071
171.0 / 303.071
171.5 / 303.071
172.0 / 303.071
172.5 / 303.071
173.0 / 303.071
173.5 / 303.071
174.0 / 303.071
174.5 / 303.071
175.0 / 303.071
175.5 / 303.071
176.0 / 303.071
176.5 / 303.071
177.0 / 303.071
177.5 / 303.071
178.0 / 303.071
178.5 / 303.071
179.0 / 303.071
179.5 / 303.071
180.0 / 303.071
180.5 / 303.071
181.0 / 303.071
181.5 / 303.071
182.0 / 303.071
182.5 / 303.071
183.0 / 303.071
183.5 / 303.071
184.0 / 303.071
184.5 / 303.071
185.0 / 303.071
185.5 / 303.071
186.0 / 303.071
186.5 / 303.071
187.0 / 303.071
187.5 / 303.071
188.0 / 303.071
188.5 / 303.071
189.0 / 303.071
189.5 / 303.071
190.0 / 303.071
190.5 / 303.071
191.0 / 303.071
191.5 / 303.071
192.0 / 303.071
192.5 / 303.071
193.0 / 303.071
193.5 / 303.071
194.0 / 303.071
194.5 / 303.071
195.0 / 303.071
195.5 / 303.071
196.0 / 303.071
196.5 / 303.071
197.0 / 303.071
197.5 / 303.071
198.0 / 303.071
198.5 / 303.071
199.0 / 303.071
199.5 / 

121.5 / 232.8529
122.0 / 232.8529
122.5 / 232.8529
123.0 / 232.8529
123.5 / 232.8529
124.0 / 232.8529
124.5 / 232.8529
125.0 / 232.8529
125.5 / 232.8529
126.0 / 232.8529
126.5 / 232.8529
127.0 / 232.8529
127.5 / 232.8529
128.0 / 232.8529
128.5 / 232.8529
129.0 / 232.8529
129.5 / 232.8529
130.0 / 232.8529
130.5 / 232.8529
131.0 / 232.8529
131.5 / 232.8529
132.0 / 232.8529
132.5 / 232.8529
133.0 / 232.8529
133.5 / 232.8529
134.0 / 232.8529
134.5 / 232.8529
135.0 / 232.8529
135.5 / 232.8529
136.0 / 232.8529
136.5 / 232.8529
137.0 / 232.8529
137.5 / 232.8529
138.0 / 232.8529
138.5 / 232.8529
139.0 / 232.8529
139.5 / 232.8529
140.0 / 232.8529
140.5 / 232.8529
141.0 / 232.8529
141.5 / 232.8529
142.0 / 232.8529
142.5 / 232.8529
143.0 / 232.8529
143.5 / 232.8529
144.0 / 232.8529
144.5 / 232.8529
145.0 / 232.8529
145.5 / 232.8529
146.0 / 232.8529
146.5 / 232.8529
147.0 / 232.8529
147.5 / 232.8529
148.0 / 232.8529
148.5 / 232.8529
149.0 / 232.8529
149.5 / 232.8529
150.0 / 232.8529
150.5 / 232.85

...

43.5 / 266.8449
44.0 / 266.8449
44.5 / 266.8449
45.0 / 266.8449
45.5 / 266.8449
46.0 / 266.8449
46.5 / 266.8449
47.0 / 266.8449
47.5 / 266.8449
48.0 / 266.8449
48.5 / 266.8449
49.0 / 266.8449
49.5 / 266.8449
50.0 / 266.8449
50.5 / 266.8449
51.0 / 266.8449
51.5 / 266.8449
52.0 / 266.8449
52.5 / 266.8449
53.0 / 266.8449
53.5 / 266.8449
54.0 / 266.8449
54.5 / 266.8449
55.0 / 266.8449
55.5 / 266.8449
56.0 / 266.8449
56.5 / 266.8449
57.0 / 266.8449
57.5 / 266.8449
58.0 / 266.8449
58.5 / 266.8449
59.0 / 266.8449
59.5 / 266.8449
60.0 / 266.8449
60.5 / 266.8449
61.0 / 266.8449
61.5 / 266.8449
62.0 / 266.8449
62.5 / 266.8449
63.0 / 266.8449
63.5 / 266.8449
64.0 / 266.8449
64.5 / 266.8449
65.0 / 266.8449
65.5 / 266.8449
66.0 / 266.8449
66.5 / 266.8449
67.0 / 266.8449
67.5 / 266.8449
68.0 / 266.8449
68.5 / 266.8449
69.0 / 266.8449
69.5 / 266.8449
70.0 / 266.8449
70.5 / 266.8449
71.0 / 266.8449
71.5 / 266.8449
72.0 / 266.8449
72.5 / 266.8449
73.0 / 266.8449
73.5 / 266.8449
74.0 / 266.8449
74.5 / 2

24.0 / 410.3377
24.5 / 410.3377
25.0 / 410.3377
25.5 / 410.3377
26.0 / 410.3377
26.5 / 410.3377
27.0 / 410.3377
27.5 / 410.3377
28.0 / 410.3377
28.5 / 410.3377
29.0 / 410.3377
29.5 / 410.3377
30.0 / 410.3377
30.5 / 410.3377
31.0 / 410.3377
31.5 / 410.3377
32.0 / 410.3377
32.5 / 410.3377
33.0 / 410.3377
33.5 / 410.3377
34.0 / 410.3377
34.5 / 410.3377
35.0 / 410.3377
35.5 / 410.3377
36.0 / 410.3377
36.5 / 410.3377
37.0 / 410.3377
37.5 / 410.3377
38.0 / 410.3377
38.5 / 410.3377
39.0 / 410.3377
39.5 / 410.3377
40.0 / 410.3377
40.5 / 410.3377
41.0 / 410.3377
41.5 / 410.3377
42.0 / 410.3377
42.5 / 410.3377
43.0 / 410.3377
43.5 / 410.3377
44.0 / 410.3377
44.5 / 410.3377
45.0 / 410.3377
45.5 / 410.3377
46.0 / 410.3377
46.5 / 410.3377
47.0 / 410.3377
47.5 / 410.3377
48.0 / 410.3377
48.5 / 410.3377
49.0 / 410.3377
49.5 / 410.3377
50.0 / 410.3377
50.5 / 410.3377
51.0 / 410.3377
51.5 / 410.3377
52.0 / 410.3377
52.5 / 410.3377
53.0 / 410.3377
53.5 / 410.3377
54.0 / 410.3377
54.5 / 410.3377
55.0 / 4

269.5 / 410.3377
270.0 / 410.3377
270.5 / 410.3377
271.0 / 410.3377
271.5 / 410.3377
272.0 / 410.3377
272.5 / 410.3377
273.0 / 410.3377
273.5 / 410.3377
274.0 / 410.3377
274.5 / 410.3377
275.0 / 410.3377
275.5 / 410.3377
276.0 / 410.3377
276.5 / 410.3377
277.0 / 410.3377
277.5 / 410.3377
278.0 / 410.3377
278.5 / 410.3377
279.0 / 410.3377
279.5 / 410.3377
280.0 / 410.3377
280.5 / 410.3377
281.0 / 410.3377
281.5 / 410.3377
282.0 / 410.3377
282.5 / 410.3377
283.0 / 410.3377
283.5 / 410.3377
284.0 / 410.3377
284.5 / 410.3377
285.0 / 410.3377
285.5 / 410.3377
286.0 / 410.3377
286.5 / 410.3377
287.0 / 410.3377
287.5 / 410.3377
288.0 / 410.3377
288.5 / 410.3377
289.0 / 410.3377
289.5 / 410.3377
290.0 / 410.3377
290.5 / 410.3377
291.0 / 410.3377
291.5 / 410.3377
292.0 / 410.3377
292.5 / 410.3377
293.0 / 410.3377
293.5 / 410.3377
294.0 / 410.3377
294.5 / 410.3377
295.0 / 410.3377
295.5 / 410.3377
296.0 / 410.3377
296.5 / 410.3377
297.0 / 410.3377
297.5 / 410.3377
298.0 / 410.3377
298.5 / 410.33

107.5 / 325.6206
108.0 / 325.6206
108.5 / 325.6206
109.0 / 325.6206
109.5 / 325.6206
110.0 / 325.6206
110.5 / 325.6206
111.0 / 325.6206
111.5 / 325.6206
112.0 / 325.6206
112.5 / 325.6206
113.0 / 325.6206
113.5 / 325.6206
114.0 / 325.6206
114.5 / 325.6206
115.0 / 325.6206
115.5 / 325.6206
116.0 / 325.6206
116.5 / 325.6206
117.0 / 325.6206
117.5 / 325.6206
118.0 / 325.6206
118.5 / 325.6206
119.0 / 325.6206
119.5 / 325.6206
120.0 / 325.6206
120.5 / 325.6206
121.0 / 325.6206
121.5 / 325.6206
122.0 / 325.6206
122.5 / 325.6206
123.0 / 325.6206
123.5 / 325.6206
124.0 / 325.6206
124.5 / 325.6206
125.0 / 325.6206
125.5 / 325.6206
126.0 / 325.6206
126.5 / 325.6206
127.0 / 325.6206
127.5 / 325.6206
128.0 / 325.6206
128.5 / 325.6206
129.0 / 325.6206
129.5 / 325.6206
130.0 / 325.6206
130.5 / 325.6206
131.0 / 325.6206
131.5 / 325.6206
132.0 / 325.6206
132.5 / 325.6206
133.0 / 325.6206
133.5 / 325.6206
134.0 / 325.6206
134.5 / 325.6206
135.0 / 325.6206
135.5 / 325.6206
136.0 / 325.6206
136.5 / 325.62

KeyboardInterrupt: 

In [15]:
df_train[(df_train['Time'] >= 0) & (df_train['Time'] <= 0 + 2.5)]

,Time,Data,Arousal,Dominance,Valence,Emotion
0,0.000000,-0.010864,-0.002000,0.000000,-0.002500,disgust
1,0.000063,-0.012909,-0.002000,0.000000,-0.002500,disgust
2,0.000125,-0.009735,-0.002000,0.000000,-0.002500,disgust
3,0.000188,-0.004852,-0.002000,0.000000,-0.002500,disgust
4,0.000250,-0.006348,-0.002000,0.000000,-0.002500,disgust
...,...,...,...,...,...,...
37085,2.499750,0.004242,7.561667,20.511667,16.316667,surprise
37086,2.499812,0.010071,7.561667,20.511667,16.316667,surprise
37087,2.499875,-0.007355,7.561667,20.511667,16.316667,surprise
37088,2.499938,-0.007172,7.561667,20.511667,16.316667,surprise


In [14]:
df_train

,Time,Data,Arousal,Dominance,Valence,Emotion
0,0.000000,-0.010864,-0.002000,0.000000,-0.0025,disgust
1,0.000063,-0.012909,-0.002000,0.000000,-0.0025,disgust
2,0.000125,-0.009735,-0.002000,0.000000,-0.0025,disgust
3,0.000188,-0.004852,-0.002000,0.000000,-0.0025,disgust
4,0.000250,-0.006348,-0.002000,0.000000,-0.0025,disgust
...,...,...,...,...,...,...
4606746,325.509000,-0.000244,13.151667,27.243333,-13.9500,disgust
4606747,325.513100,-0.000244,13.151667,27.243333,-13.6850,disgust
4606748,325.566300,-0.000244,12.888333,27.243333,-13.6850,disgust
4606749,325.578900,-0.000244,12.888333,27.700000,-13.6850,disgust


In [5]:
df_features_complete

,0,1,2,3,4,5,6,7,8,9,...,156,157,158,159,160,161,Arousal,Dominance,Valence,Emotion
0,0.129173,0.633480,0.637506,0.629961,0.618175,0.614299,0.641680,0.683082,0.669334,0.663436,...,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,14.909147,11.661645,14.132956,surprise
1,0.129173,0.633480,0.637506,0.629961,0.618175,0.614299,0.641680,0.683082,0.669334,0.663436,...,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,20.141072,15.358798,20.234042,surprise
2,0.129173,0.633480,0.637506,0.629961,0.618175,0.614299,0.641680,0.683082,0.669334,0.663436,...,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,23.557137,18.376026,26.088335,joy
3,0.129173,0.633480,0.637506,0.629961,0.618175,0.614299,0.641680,0.683082,0.669334,0.663436,...,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,25.22692,21.036799,29.19959,joy
4,0.129173,0.633480,0.637506,0.629961,0.618175,0.614299,0.641680,0.683082,0.669334,0.663436,...,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,26.355407,22.062475,30.213244,joy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,0.085137,0.727574,0.716821,0.713619,0.710770,0.692016,0.691655,0.700262,0.679796,0.680309,...,0.003208,0.003125,0.003028,0.002826,0.002227,0.001602,-5.408264,-0.040454,14.488924,disgust
812,0.085137,0.727574,0.716821,0.713619,0.710770,0.692016,0.691655,0.700262,0.679796,0.680309,...,0.003208,0.003125,0.003028,0.002826,0.002227,0.001602,-4.513413,-0.31622,15.66732,disgust
813,0.085137,0.727574,0.716821,0.713619,0.710770,0.692016,0.691655,0.700262,0.679796,0.680309,...,0.003208,0.003125,0.003028,0.002826,0.002227,0.001602,-2.948888,-0.368104,16.717284,disgust
814,0.085137,0.727574,0.716821,0.713619,0.710770,0.692016,0.691655,0.700262,0.679796,0.680309,...,0.003208,0.003125,0.003028,0.002826,0.002227,0.001602,-2.12687,-0.24222,16.425597,disgust


In [ ]:
df_label

In [ ]:
df_train